![title](https://github.com/STScI-MIRI/MRS-ExampleNB/raw/main/assets/banner1.png)

### Jdaviz Imviz Demo
#### AAS 245 National Harbor, Jan 2025
Author : Clare Shanahan, Space Telescope Science Institute

#### Tutorial Overview
This tutorial will demonstrate an example workflow to show of some key features of Imviz. The example scenario here is that we are
looking at two exposures to visually inspect how well they are aligned.

1. Starting Imviz and loading data.
2. Setting display options for loaded data.
3. Aligning images by pixel / wcs. Blinking between loaded images. 
4. Loading GAIA catalog and plotting sources on images. Zooming and panning on image.
5. 

The notebook demo will demonstrate how to do these tasks in the application UI, but this notebook has API calls to do the same steps programatically. (That is the magic of jdaviz!)


First, import, create, and show an Imviz instance in the notebook. It will be empty until we load data, but the application 
displayed in the following cell will be our workspace, where all changes made by clicking or programatically via the API will be reflected. (You will be scrolling up to this cell frequently if you are following along in the notebook.)

In [1]:
from jdaviz import Imviz
imviz = Imviz()
imviz.show()

Application(config='imviz', docs_link='https://jdaviz.readthedocs.io/en/latest/imviz/index.html', events=['cal…

### Loading Data
In addition to loading local data or array data (e.g., a Spectrum1D for specviz/specviz2d, or numpy array/CCDData etc in imviz) in a notebook, jdaviz can download and load data directly from the MAST archive.

(MAST queries are done at the association level, so it is a few extra steps to get an individual exposure. The datasets we will load are the first 2 returned by this query, for those curious how we obtained the URIs in the 'load_data' cell)

In [5]:
##  comment out to see how we found the observation URIs to load
# from astroquery.mast import Observations
# obs = Observations.query_criteria(obs_id='jw01783-o003_t009_nircam_clear-f335m')
# product_list = Observations.get_product_list(obs)
## table where 'dataURI' column is what we want
# filtered_products = Observations.filter_products(product_list, productSubGroupDescription = 'CAL')

We will now download and load 2 nircam F335M images of M51, dithered exposures from the same observation.

Once 'load_data' is run, we will see these images appear in the Imviz cell above. With 'cache=True', a local copy of the data is saved so it will not need to be downloaded next time.

(Note: There is distinction to be noted here - these two images were loaded into the same viewer (imviz-0). It is also possible to create a second viewer and display the images side by side, but for the purposes of this demo we prefer them in the same viewer.)

In [3]:
# load first exposure into image viewer
imviz.load_data('mast:JWST/product/jw01783003004_02101_00003_nrcalong_cal.fits', cache=True)

# load second exposure into image viewer
imviz.load_data('mast:JWST/product/jw01783003004_02101_00001_nrcalong_cal.fits', cache=True)

INFO:astroquery:Found cached file /Users/cshanahan/packages/testing_notebooks/jdaviz/jw01783003004_02101_00003_nrcalong_cal.fits with expected size 117544320.


INFO: Found cached file /Users/cshanahan/packages/testing_notebooks/jdaviz/jw01783003004_02101_00003_nrcalong_cal.fits with expected size 117544320. [astroquery.query]


INFO:astroquery:Found cached file /Users/cshanahan/packages/testing_notebooks/jdaviz/jw01783003004_02101_00001_nrcalong_cal.fits with expected size 117544320.


INFO: Found cached file /Users/cshanahan/packages/testing_notebooks/jdaviz/jw01783003004_02101_00001_nrcalong_cal.fits with expected size 117544320. [astroquery.query]


Once these files are loaded (which will take longer the first time since they need to be downloaded), they will appear in Imviz. Each dataset, loaded (using default names A and B in this case) will appear in the data menu on the left hand side. You can select/deselect loaded data to display. To blink between images, press the 'b' key.

### Modifying Image Display Options
Now, we will modify some of the display options to better suit our data. For the live demo, we will do this in the UI in the 'Plot Options' plugin by modifying the image stretch from linear to logarithmic, and setting vmax to a more appropriate value. The following cell, when uncommented, accomplishes the same task from the API.

(Take a look around the Plot Options plugin, there is a lot more you can do to customize display settings including colormap, setting layer colors and opacities to create composite RGB images, and displaying contours.)

In [7]:
# the following code is the API equivalent to the series of UI clicks we will do in the live demo

# get the 'Plot Options' plugin
plot_options = imviz.plugins['Plot Options']

# enable mutiselect so our chosen options are applied to both images
plot_options.layer_multiselect = True

# switch stretch function from default linear to log
plot_options.stretch_function = 'log'

# use the 99.5% stretch function preset
plot_options.stretch_preset = '99.5%'

# increase vmax so we can see structures in the data more clearly
plot_options.stretch_vmax = plot_options.stretch_vmax.value * 100.

#### Linking by WCS

By default, images are pixel linked when loaded. We can link them by w